In [1]:
#   ___                  _                
#  / _/______ ____  ____(_)__ _______     
# / _/ __/ _ `/ _ \/ __/ (_-</ __/ _ \    
#/_//_/  \_,_/_//_/\__/_/___/\__/\___/    
# ___ _____(_)__ ___ ____  / /_(_)       
# / _ `/ __/ (_-</ _ `/ _ \/ __/ /        
# \_, /_/ /_/___/\_,_/_//_/\__/_/         
#/___/
#
#Samee Lab @ Baylor College Of Medicine
#francisco.grisanticanozo@bcm.edu

In [47]:
library(scPred)
library(Seurat)


library(dplyr)
library(Matrix)
library(SeuratDisk)
library(methods)
library(ggplot2)
library(tidyverse)
require(SingleCellExperiment)
library(doParallel)
library(preprocessCore)

options(stringsAsFactors = FALSE)
future::plan("multiprocess", workers = 20) # do parallel
options(future.globals.maxSize = 20000 * 1024^2)

In [16]:
file_path_train_data <- '../Data/Single_Cell_Reference/adata_processed_for_rds.h5seurat'
file_path_train_data_obs <- '../Data/Single_Cell_Reference/adata_metadata.csv'

file_path_predict_data <- '../Data/Merfish/merfish_for_rds.h5seurat'
file_path_predict_data_obs <- '../Data/Merfish/merfish_metadata.csv'

In [40]:
# load train data

adata_train <- LoadH5Seurat(file_path_train_data)
adata_train_obs <- as.data.frame(read_csv(file_path_train_data_obs))
adata_train_m <- t(as.matrix(GetAssayData(adata_train, assay= 'RNA', slot = "data")))
adata_train <- CreateSeuratObject(t(adata_train_m))
adata_train$celltype <- adata_train_obs$celltype

# load predict data

adata_predict <- LoadH5Seurat(file_path_predict_data)
adata_predict_obs <- as.data.frame(read_csv(file_path_predict_data_obs))
adata_predict_m <- t(as.matrix(GetAssayData(adata_predict, assay= 'RNA', slot = "data")))
adata_predict <- CreateSeuratObject(t(adata_predict_m))
adata_predict$celltype <- adata_predict_obs$celltype

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results


── Column specification ────────────────────────────────────────────────────────
cols(
  `Cell name` = col_character(),
  Sex = col_character(),
  `Replicate number` = col_double(),
  `Cell class (determined from clustering of all cells)` = col_character(),
  `Non-neuronal cluster (determined from clustering of all cells)` = col_character(),
  `Neuronal cluster (determined from clustering of inhibitory or excitatory neurons)` = col_character(),
  n_genes_by_counts = col_double(),
  total_counts = col_double(),
  total_counts_mt = col_double(),
  pct_counts_mt = col_double(),
  celltype = col_character()
)


Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-le

# Run scPRED

## Train model

In [42]:
adata_train <- adata_train  %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

Centering and scaling data matrix

PC_ 1 
Positive:  Syt4, Scg2, Pak3, Gad1, Gabra1, Nnat, Gda, Cpne5, Gabrg1, Vgf 
	   Rgs2, Prlr, Ar, Htr2c, Baiap2, Oprl1, Irs4, Col25a1, Cspg5, Pnoc 
	   Ntng1, Isl1, Cartpt, Pgr, Penk, Necab1, Slc17a6, Nts, Cbln2, Oprk1 
Negative:  Ermn, Mbp, Opalin, Gjc3, Lpar1, Ndrg1, Etv1, Ttyh2, Sgk1, Plin3 
	   Sox8, Klf4, Sema4d, Cdkn1a, Fos, Slco1a4, Selplg, Slc15a3, Pdgfra, Fn1 
	   Mlc1, Aqp4, Aldh1l1, Traf4, Cyr61, Cenpe, Rgs5, Bmp7, Ndnf, Man1a 
PC_ 2 
Positive:  Pdgfra, Sox6, Traf4, Cyr61, Cspg5, Sox4, Cxcl14, Cdkn1a, Fos, Baiap2 
	   Nnat, Bmp7, Klf4, Tiparp, Aqp4, Mlc1, Tac1, Slco1a4, Ccnd2, Fn1 
	   Adora2a, Selplg, Ano3, Slc15a3, Egr2, Sox8, Penk, Cpne5, Pou3f2, Gem 
Negative:  Prlr, Ermn, Slc17a6, Opalin, Pak3, Irs4, Mbp, Lpar1, Ndrg1, Ar 
	   Adcyap1, Gabra1, Cbln2, Pnoc, Dgkk, Bdnf, Cbln1, Nos1, Syt4, Pgr 
	   Gal, Ttyh2, Trh, Esr1, Onecut2, Etv1, Oprl1, Arhgap36, Amigo2, Col25a1 
PC_ 3 
Positive:  Cspg5, Pdgfra, Traf4, Sox6, Slc17a6, Pcdh11x, Sox

In [43]:
adata_train <- getFeatureSpace(adata_train, "celltype")

●  Extracting feature space for each cell type...
DONE!


In [44]:
cl <- makePSOCKcluster(10)
registerDoParallel(cl)
adata_train <- trainModel(adata_train, seed = 123,allowParallel = TRUE)
stopCluster(cl)

●  Training models for each cell type...
DONE!


ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function ‘get_scpred’ for signature ‘"function"’


In [45]:
get_scpred(adata_train)

'scPred' object
✔  Prediction variable = celltype 
✔  Discriminant features per cell type
✔  Training model(s)
Summary

|Cell type                |     n| Features|Method    |   ROC|  Sens|  Spec|
|:------------------------|-----:|--------:|:---------|-----:|-----:|-----:|
|Astrocytes               |   857|       50|svmRadial | 0.985| 0.798| 0.999|
|Endothelial              |   541|       50|svmRadial | 0.991| 0.932| 1.000|
|Ependymal                |    69|       50|svmRadial | 0.966| 0.508| 1.000|
|Excitatory               |  3511|       50|svmRadial | 0.989| 0.881| 0.990|
|Immature oligodendrocyte |  1692|       50|svmRadial | 0.999| 0.947| 0.998|
|Inhibitory               | 15042|       50|svmRadial | 0.993| 0.979| 0.962|
|Mature oligodendrocyte   |  6707|       50|svmRadial | 1.000| 0.993| 0.999|
|Microglia                |   828|       50|svmRadial | 0.999| 0.972| 1.000|

## Classify on predict data

In [48]:
adata_predict <- adata_predict %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

Centering and scaling data matrix

PC_ 1 
Positive:  Ttyh2, Mbp, Ermn, Sgk1, Gjc3, Ndrg1, Lpar1, Opalin, Plin3, Sox8 
	   Fn1, Aqp4, Rgs5, Slco1a4, Cck, Klf4, Aldh1l1, Mlc1, Gnrh1, Cxcl14 
	   Gbx2, Igf1r, Cdkn1a, Adora2a, Cyr61, Pdgfra, Etv1, Ndnf, Bmp7, Selplg 
Negative:  Pak3, Syt4, Oprl1, Gpr165, Ar, Irs4, Cbln2, Prlr, Gabra1, Gad1 
	   Gda, Glra3, Pgr, Nos1, Dgkk, Col25a1, Tmem108, Amigo2, Htr2c, Slc17a6 
	   Scg2, Rgs2, Arhgap36, Cpne5, Esr1, Npy2r, Pnoc, Igf2r, Cspg5, Rnd3 
PC_ 2 
Positive:  Ermn, Mbp, Sgk1, Gjc3, Opalin, Ndrg1, Lpar1, Sema4d, Ttyh2, Omp 
	   Glra3, Man1a, Sox8, Pak3, Prlr, Plin3, Esr1, Trhr, Kiss1r, Scg2 
	   Oprl1, Syt4, Ar, Col25a1, Coch, Tacr1, Pgr, Irs4, Gda, Synpr 
Negative:  Mlc1, Aldh1l1, Aqp4, Cxcl14, Fn1, Rgs5, Igf1r, Slco1a4, Pou3f2, Klf4 
	   Sox6, Cyr61, Bmp7, Cspg5, Sema3c, Nnat, Traf4, Myh11, Adora2a, Cdkn1a 
	   Ndnf, Ace2, Baiap2, Ccnd2, Pdgfra, Sox4, Tiparp, Lepr, Sln, Lmod1 
PC_ 3 
Positive:  Aqp4, Aldh1l1, Sox8, Cxcl14, Sox6, Pou3f2, Cspg5, M

In [49]:
adata_predict <- scPredict(adata_predict, adata_train)

●  Matching reference with new dataset...
	 ─ 155 features present in reference loadings
	 ─ 155 features shared between reference and new dataset
	 ─ 100% of features in the reference are present in new dataset
●  Aligning new data to reference...


Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 44701000)”
Harmony 1/20

Harmony 2/20

Harmony converged after 2 iterations



●  Classifying cells...
DONE!


In [50]:
adata_predict@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,celltype,scpred_Excitatory,scpred_Inhibitory,scpred_Mature.oligodendrocyte,scpred_Endothelial,scpred_Immature.oligodendrocyte,scpred_Microglia,scpred_Astrocytes,scpred_Ependymal,scpred_max,scpred_prediction,scpred_no_rejection
,<fct>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
6749ccb4-2ed1-4029-968f-820a287f43c8,SeuratProject,58.04731,44,Astrocytes,0.003972798,0.0859774465,0.0001414578,0.0009400796,0.0211416628,1.228524e-04,0.867969215,0.0015235583,0.8679692,Astrocytes,Astrocytes
6cac74bd-4ea7-4701-8701-42563cc65eb8,SeuratProject,81.04589,73,Inhibitory,0.159330634,0.7542433436,0.0006946783,0.0109751574,0.0115072640,2.489002e-03,0.019172231,0.0022664842,0.7542433,Inhibitory,Inhibitory
9f29bd57-16a5-4b26-b9f5-37598809da9e,SeuratProject,68.36556,47,Inhibitory,0.168195078,0.7663067743,0.0007283751,0.0097085353,0.0101149303,2.151863e-03,0.020724093,0.0020451206,0.7663068,Inhibitory,Inhibitory
d7eb4e0b-276e-47e3-a55c-0b033180a2fe,SeuratProject,60.06772,58,Inhibitory,0.156895132,0.7542523687,0.0006784531,0.0107844824,0.0117387921,2.448396e-03,0.020142088,0.0022680011,0.7542524,Inhibitory,Inhibitory
54434f3a-eba9-4aec-af35-c9d317ffa1d5,SeuratProject,58.92851,36,Inhibitory,0.151631126,0.7286747110,0.0006445768,0.0094439657,0.0088746623,2.265386e-03,0.024089234,0.0020943305,0.7286747,Inhibitory,Inhibitory
6dcfb0ba-763f-4330-8ff6-aa55fd5d6ba6,SeuratProject,56.65099,40,Mature oligodendrocyte,0.014865791,0.6311885549,0.0147537912,0.0011627880,0.0160601006,4.325456e-03,0.004810699,0.0005165229,0.6311886,Inhibitory,Inhibitory
45fbf54d-35d2-4ac3-a42a-c59751e9d4f7,SeuratProject,81.24770,92,Inhibitory,0.162132412,0.7760338073,0.0007199848,0.0081473773,0.0105186842,2.025501e-03,0.016335308,0.0017214148,0.7760338,Inhibitory,Inhibitory
cb0b8dcf-a941-4766-bbc9-9e0271392c43,SeuratProject,93.97102,93,Inhibitory,0.168038655,0.7407204014,0.0007406734,0.0105471107,0.0116823938,2.396222e-03,0.017967759,0.0021502018,0.7407204,Inhibitory,Inhibitory
b288e78c-3693-4aad-b40f-a21ad62be583,SeuratProject,64.67680,63,Inhibitory,0.128641003,0.8215523498,0.0006926745,0.0097030504,0.0124579132,2.442773e-03,0.017964170,0.0019293388,0.8215523,Inhibitory,Inhibitory


In [52]:
write.csv(adata_predict@meta.data,'../outputs/scPRED_predictions.csv')